In [2]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() ### Load tensorFlow

#import tensorflow as tf
#tf.reset_default_graph()
tf.set_random_seed(42)

Instructions for updating:
non-resource variables are not supported in the long term


### Collect Data

In [3]:
# Read hdf5 file having the dataset

import numpy as np
import h5py

hf = h5py.File('SVHN_single_grey1.h5', 'r')

print(hf.keys())

<KeysViewHDF5 ['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']>


In [3]:
# # normalize inputs from 0-255 to 0-1
X_train = hf.get('X_train')
trainX = np.array(X_train)

# normalize inputs from 0-255 to 0-1
trainX = trainX / 255.0

In [4]:
y_train = hf.get('y_train')
trainY = np.array(y_train)
trainY.shape

(42000,)

In [5]:
X_test = hf.get('X_test')
testX = np.array(X_test)

# normalize inputs from 0-255 to 0-1
testX = testX / 255.0

In [6]:
y_test = hf.get('y_test')
testY = np.array(y_test)
testY.shape

(18000,)

In [1]:
hf.close()

NameError: name 'hf' is not defined

In [7]:
trainX.shape

(42000, 32, 32)

In [8]:
trainY.shape

(42000,)

In [9]:
#One hot encode the train and test targets
trainY = tf.keras.utils.to_categorical(trainY, num_classes=10)
testY = tf.keras.utils.to_categorical(testY, num_classes=10)

In [10]:
trainY.shape

(42000, 10)

In [11]:
trainY.shape

(42000, 10)

## Build the Graph

In [12]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32,)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [13]:
#Create optimizer with non-default learning rate

# Run 1 - Test Accuracy - 78%
#sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)


# Run 2 - Test Accuracy - 84%
#adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.03)

# Run 3 - Test Accuracy - 87%
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)


#Compile the model
model.compile(optimizer = adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

## Review model

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dense (Dense)                (None, 200)               205000    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0

## Train the model

In [15]:
model.fit(trainX,trainY, validation_data=(testX,testY), epochs=30, batch_size=64)

Train on 42000 samples, validate on 18000 samples
Epoch 1/30
42000/42000 [==============================] - 14s 344us/sample - loss: 1.7721 - acc: 0.3990 - val_loss: 1.0677 - val_acc: 0.6723
Epoch 2/30
42000/42000 [==============================] - 14s 340us/sample - loss: 1.2140 - acc: 0.6110 - val_loss: 0.9128 - val_acc: 0.7175
Epoch 3/30
42000/42000 [==============================] - 14s 344us/sample - loss: 1.0652 - acc: 0.6644 - val_loss: 0.7860 - val_acc: 0.7639
Epoch 4/30
42000/42000 [==============================] - 15s 348us/sample - loss: 0.9792 - acc: 0.6888 - val_loss: 0.7222 - val_acc: 0.7803
Epoch 5/30
42000/42000 [==============================] - 15s 354us/sample - loss: 0.9181 - acc: 0.7114 - val_loss: 0.6743 - val_acc: 0.7986
Epoch 6/30
42000/42000 [==============================] - 14s 322us/sample - loss: 0.8663 - acc: 0.7266 - val_loss: 0.6757 - val_acc: 0.7911
Epoch 7/30
42000/42000 [==============================] - 14s 328us/sample - loss: 0.8254 - acc: 0.7405 

In [16]:
import matplotlib.pyplot as plt
d = model.history.history

plt.plot(d['acc'])
plt.plot(d['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(d['loss'])
plt.plot(d['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show() 

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [17]:
results = model.evaluate(testX, testY)


18000/18000 [==============================] - 2s 139us/sample - loss: 0.4621 - acc: 0.8621


In [18]:
predY_cls = model.predict_classes(testX, batch_size=200, verbose=0)

In [19]:
print(predY_cls)

[0 7 2 ... 7 9 2]


In [20]:
print(np.array(y_test))

[1 7 2 ... 7 9 2]


In [21]:
from sklearn.metrics import confusion_matrix, f1_score, classification_report

In [22]:
confusion_matrix(np.array(y_test), predY_cls)

array([[1636,   24,    5,   13,   18,    5,   23,   20,   19,   51],
       [  30, 1624,   14,   23,   45,    9,   16,   24,   18,   25],
       [  18,   31, 1530,   37,   26,   15,    7,   57,   25,   57],
       [  10,   37,   27, 1424,   24,   98,   14,   21,   37,   27],
       [  19,   54,   18,   13, 1607,   16,   18,    9,    8,   50],
       [   9,   17,    7,   87,   10, 1503,   54,    8,   32,   41],
       [  30,   24,   13,   22,   30,   57, 1567,    6,   55,   28],
       [  22,   66,   29,   27,   18,   10,    7, 1590,   10,   29],
       [  22,   43,   14,   44,   25,   29,   87,    8, 1474,   66],
       [  35,   37,   13,   35,   17,   44,   15,   18,   27, 1563]],
      dtype=int64)

In [23]:
f1_score(np.array(y_test), predY_cls, average='weighted')

0.862251514979578

In [24]:
classification_report(np.array(y_test), predY_cls, output_dict=True )

{'0': {'precision': 0.8935008192244676,
  'recall': 0.9018743109151047,
  'f1-score': 0.8976680384087791,
  'support': 1814},
 '1': {'precision': 0.8298415942769545,
  'recall': 0.888402625820569,
  'f1-score': 0.8581241743725231,
  'support': 1828},
 '2': {'precision': 0.9161676646706587,
  'recall': 0.848585690515807,
  'f1-score': 0.8810826374892023,
  'support': 1803},
 '3': {'precision': 0.8255072463768116,
  'recall': 0.8283885980221058,
  'f1-score': 0.8269454123112661,
  'support': 1719},
 '4': {'precision': 0.8829670329670329,
  'recall': 0.8868653421633554,
  'f1-score': 0.8849118942731278,
  'support': 1812},
 '5': {'precision': 0.841545352743561,
  'recall': 0.8501131221719457,
  'f1-score': 0.8458075407990996,
  'support': 1768},
 '6': {'precision': 0.8667035398230089,
  'recall': 0.855349344978166,
  'f1-score': 0.860989010989011,
  'support': 1832},
 '7': {'precision': 0.9028960817717206,
  'recall': 0.879424778761062,
  'f1-score': 0.8910058840011208,
  'support': 1808}